<a href="https://colab.research.google.com/github/leeseungchae/watching_you_chatbot/blob/SeungChae/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-e564pzhl
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-e564pzhl
     |████████████████████████████████| 132 kB 26.9 MB/s 
     |████████████████████████████████| 344 kB 70.2 MB/s 
     |████████████████████████████████| 49.1 MB 1.7 MB/s 
     |████████████████████████████████| 4.5 MB 52.2 MB/s 
     |████████████████████████████████| 1.2 MB 69.4 MB/s 
     |████████████████████████████████| 4.2 MB 73.3 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 91.2 MB/s 
     |████████████████████████████████| 8.8 MB 88.2 MB/s 
     |████████████████████████████████| 79 kB 10.6 MB/s 
     |████████████████████████████████| 127 kB 97.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=4457fe9378c

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split 

In [4]:
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [88]:
csv = pd.read_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing.csv') 
trainData, testData = train_test_split(csv, test_size = 0.3, random_state=42) 

In [90]:
df1 = pd.DataFrame(trainData)
df1.to_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_train.tsv',index=False,
               sep='\t',encoding='utf-8')
df2 = pd.DataFrame(testData)
df2.to_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_test.tsv',index=False,
               sep='\t',encoding='utf-8')



In [91]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_train.tsv", field_indices=[0,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing_test.tsv", field_indices=[0,2], num_discard_samples=1)

In [92]:
dataset_train[0]

['그리고 차가운 것으로 주세요', '0']

In [94]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [95]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [96]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [97]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [98]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [99]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=126,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [100]:
device = torch.device("cuda:0")
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [101]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [102]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [103]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [104]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [105]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [106]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/34 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.061009407043457 train acc 0.0
epoch 1 train acc 0.09718574929971989


  0%|          | 0/15 [00:00<?, ?it/s]

epoch 1 test acc 0.6056469298245614


  0%|          | 0/34 [00:00<?, ?it/s]

...

epoch 14 test acc 0.8617324561403509


  0%|          | 0/34 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.5514253377914429 train acc 0.953125
epoch 15 train acc 0.9218531162464986


  0%|          | 0/15 [00:00<?, ?it/s]

epoch 15 test acc 0.8617324561403509


In [107]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/03_DeepLearning/bert.pt')


In [109]:
!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
# dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing.tsv", field_indices=[0,2], num_discard_samples=1)

--2022-05-19 07:10:30--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.148.75
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.148.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  14.2MB/s    in 1.0s    

2022-05-19 07:10:31 (14.2 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]



In [25]:
# for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):

In [26]:
new_test = nlp.data.TSVDataset(".cache/ratings_train.txt" , field_indices=[1,2], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [55]:
# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)): 
#   token_ids = token_ids.long().to(device) 
#   segment_ids = segment_ids.long().to(device) 
#   valid_length= valid_length 
#   out = model(token_ids, valid_length, segment_ids)
#   prediction = out.cpu().detach().numpy().argmax()

#   batch_id = int(batch_id)
#   # prediction = float(prediction)

#   print(f"{batch_id }번째 문장의 분류 예측값은 {prediction} 입니다.")
#   # print(f"{batch_id} "+"번째 문장의 분류 예측값은"+" {prediction}"+"입니다.")

#   # print(f"{batch_id} "+"번째 문장의 분류 예측값은"+ {prediction} "+" 입니다.")

  0%|          | 0/48 [00:00<?, ?it/s]

0+번째 문장의 분류 예측값은 3788 입니다.
0 번째 문장의 분류 예측값은 {prediction}입니다.
1+번째 문장의 분류 예측값은 4166 입니다.
1 번째 문장의 분류 예측값은 {prediction}입니다.
2+번째 문장의 분류 예측값은 6434 입니다.
2 번째 문장의 분류 예측값은 {prediction}입니다.
3+번째 문장의 분류 예측값은 1142 입니다.
3 번째 문장의 분류 예측값은 {prediction}입니다.
4+번째 문장의 분류 예측값은 4544 입니다.
4 번째 문장의 분류 예측값은 {prediction}입니다.
5+번째 문장의 분류 예측값은 7820 입니다.
5 번째 문장의 분류 예측값은 {prediction}입니다.
6+번째 문장의 분류 예측값은 5552 입니다.
6 번째 문장의 분류 예측값은 {prediction}입니다.
7+번째 문장의 분류 예측값은 8 입니다.
7 번째 문장의 분류 예측값은 {prediction}입니다.
8+번째 문장의 분류 예측값은 7064 입니다.
8 번째 문장의 분류 예측값은 {prediction}입니다.
9+번째 문장의 분류 예측값은 7442 입니다.
9 번째 문장의 분류 예측값은 {prediction}입니다.
10+번째 문장의 분류 예측값은 4418 입니다.
10 번째 문장의 분류 예측값은 {prediction}입니다.
11+번째 문장의 분류 예측값은 3158 입니다.
11 번째 문장의 분류 예측값은 {prediction}입니다.
12+번째 문장의 분류 예측값은 5678 입니다.
12 번째 문장의 분류 예측값은 {prediction}입니다.
13+번째 문장의 분류 예측값은 6560 입니다.
13 번째 문장의 분류 예측값은 {prediction}입니다.
14+번째 문장의 분류 예측값은 3284 입니다.
14 번째 문장의 분류 예측값은 {prediction}입니다.
15+번째 문장의 분류 예측값은 7946 입니다.
15 번째 문장의 분류 예측값은 {prediction}입니다.
16+번째 문장의 분류 예측

In [110]:
df = pd.read_csv('/content/drive/MyDrive/Watching_You_PJ/project_ChatBot/02_preprocessing/bert_preprocessing.csv')
df

,SENTENCE,MAIN,num
0,아이스아메리카노 하나요,일반주문,0
1,네 텀블러에 넣어주세요,텀블러사용,1
2,그란데 사이즈로 주세요,일반주문,0
3,저 카푸치노로 주문할게요,일반주문,0
4,그럼 2000원 추가해서 브레드도 같이 시킬게요,결제요청,2
...,...,...,...
3043,하우스 블렌딩은 뭐예요?,메뉴문의,8
3044,약간 산미 나는 걸로 한 잔 주실래요?,일반주문,0
3045,마시고 가면 얼마예요?,일반주문,0
3046,영수증 하나 주실래요?,영수증요청,45


In [111]:
category_list = list(df['MAIN'].unique())
category_list
cat_len = len(category_list)

In [188]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        prediction = out.cpu().detach().numpy().argmax()

        test_eval=[]
        test_eval_2=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
        for k , c in enumerate(category_list):
        # for k,(c,p) in enumerate(zip(category_list,prediction)):
            
            if np.argmax(logits) == k:
                value = logits[k]
                value  = round(value*10)
                if value <50 :
                    print("죄송합니다 다시 말씀해주세요")
                    print(value)
                    return False
                else :
                    test_eval.append(c)
                    value = logits[k]
                    test_eval_2.append(value)

            # elif np.argmax(logits) == 1:
            #     test_eval.append("텀블라사용")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        # print(value)
        print(test_eval_2[0])

        # print(">> 입력하신 내용에서 " +  test_eval_2[0] + " 느껴집니다.")

# predict('안녕하세요')


using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [129]:
value = logits[k]+print(type(prediction))

<class 'numpy.int64'>


In [ ]:
prediction = out.cpu().detach().numpy().argmax()


In [189]:

end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        
        break
    predict(sentence)


    print("\n")

하고싶은 말을 입력해주세요 : 야


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 메뉴문의 느껴집니다.
5.6723647


하고싶은 말을 입력해주세요 : 왜
>> 입력하신 내용에서 메뉴문의 느껴집니다.
5.9141707


하고싶은 말을 입력해주세요 : 오오
>> 입력하신 내용에서 일반주문 느껴집니다.
8.465584


하고싶은 말을 입력해주세요 : 라떼
>> 입력하신 내용에서 일반주문 느껴집니다.
8.593819




KeyboardInterrupt: ignored